In [1]:
%load_ext autoreload 
%autoreload 2

%aimport -pandas
%aimport -scipy.stats
%aimport -numpy
%aimport -matplotlib.pyplot


import default_calcs as default
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas

In [ ]:
default.default_rate_normal_dist(-2,.3,.01)

In [ ]:
a  = stats.norm.ppf(.01)

In [ ]:
# x = effects of economic driver
x = np.linspace(-3,3,100)

In [ ]:
x_dist = stats.norm.pdf(x)

In [ ]:
plt.plot(x,a_dist)
plt.show()

In [ ]:
defaults= default.default_rate_normal_dist(x,.5,.01)

In [ ]:
plt.plot(x,defaults)
plt.show()

In [ ]:
# cum default probability
cum_def = default.inv_default_rate_normal_dist(.01,.3,defaults)

In [ ]:
df = pandas.DataFrame(data=[defaults,(cum_def)]).T
df.columns=['defaults','cum_def']

In [ ]:
df.tail()

In [ ]:
plt.plot(defaults,x_dist.cumsum())
plt.show()

In [ ]:
len(cum_def)

In [ ]:
import prepayment_calcs as pc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
fast_smm = np.random.randint(50,100,size=60)/1000

In [ ]:
a = pc.prepayment_curve_from_passive_active_composition(.09, .8, .01, .2, 60)

In [ ]:
a.plot(a.index,['fast_amount','pool_cpr'])
plt.grid()
plt.show()

In [ ]:
a['total'] = a.fast_amount + a.slow_amount

In [ ]:
a.head()

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
USD_LIBOR = BeautifulSoup(requests.get("http://www.global-rates.com/interest-rates/libor/american-dollar/american-dollar.aspx").text,
                         "html.parser")

In [ ]:
USD_LIBOR.title

In [ ]:
balances = np.array([[50000],[100000],[150000],[200000],[250000]])
coupon = np.array([0.04,0.06,0.08,0.10,0.12,.14])

In [ ]:
df = pd.DataFrame(np.pmt(rate=coupon/12,nper=360,pv=balances)).T

In [ ]:
np.arange(1,10)

In [ ]:
nper=360
pv=400000
cpn=0.055/12
df = pd.DataFrame(data=np.zeros([nper+1,1]),columns=['scheduled_balance'],
                 index =[np.arange(0,nper+1)])
df.loc[0,'scheduled_balance'] = pv

In [ ]:
def scheduled_balances(rate, nper, pv):
    df = pd.DataFrame(data=np.zeros([nper + 1, 1]), columns=['scheduled_balance'],
                      index=[np.arange(0, nper + 1)])
    df.loc[0, 'scheduled_balance'] = pv

    df['bal_percent'] = (1 - ((((1 + rate) ** np.array(df.index)) - 1) /
                           (((1 + rate) ** df.index.max()) - 1)))

    df.loc[:, 'scheduled_balance'] = df['bal_percent'] * df.loc[0,'scheduled_balance']

    return df

In [ ]:
df1 = scheduled_balances(0.055/12,360,400000)

In [ ]:
def scheduled_balance_percent(rate, nper, age):
    return (1 - ((((1 + rate) ** age) - 1) /
          (((1 + rate) ** nper) - 1)))

In [ ]:
scheduled_balance_percent(0.06/12,360,60)

In [ ]:
coupon = list([0.06, 0.08, 0.10])
age = list([60,120,180,240,300,359])

In [ ]:
results = []
for i in coupon:
    column = []
    for j in age:
        column.append(scheduled_balance_percent(i/12, 360, j))
    results.append(column)

In [ ]:
df = pd.DataFrame(results).T
df.columns=coupon
df.index=age

In [ ]:
df

In [ ]:
df = pd.DataFrame(data=np.zeros([nper + 1, 1]), columns=['scheduled_balance'],
                      index=[np.arange(0, nper + 1)])
df.loc[0, 'scheduled_balance'] = pv

In [ ]:
rate = 0.055
nper = 360

df['bal_percent'] = df.reset_index()['index'].apply(lambda x: 
                                                    scheduled_balance_percent(
                                                    rate/12, nper, x))

In [ ]:
def scheduled_balances(rate, nper, pv):
    """ Returns data frame of scheduled balances and each periods scheduled balance as a %
    of the original balance"""

    df = pd.DataFrame(data=np.zeros([nper + 1, 1]), columns=['scheduled_balance'],
                      index=[np.arange(0, nper + 1)])
    df.loc[0, 'scheduled_balance'] = pv

    df['bal_percent'] = df.reset_index()['index'].apply(lambda x:
                                                        scheduled_balance_percent(
                                                            rate / 12, nper, x))

    df.loc[:, 'scheduled_balance'] = df['bal_percent'] * df.loc[0,'scheduled_balance']

    return df

In [ ]:
def scheduled_balances(rate, nper, pv):
    """ Returns data frame of scheduled balances and each periods scheduled balance as a %
    of the original balance"""

    df = pd.DataFrame(data=np.zeros([nper + 1, 1]), columns=['scheduled_balance'],
                      index=[np.arange(0, nper + 1)])
    df.loc[0, 'scheduled_balance'] = pv
    
    df['bal_percent'] = (1 - (((1 + rate/12) ** np.array(df.index) - 1) / 
                        ((1 + rate/12) ** nper - 1)))

    df.loc[:, 'scheduled_balance'] = df['bal_percent'] * df.loc[0,'scheduled_balance']

    return df

In [ ]:
%time df = scheduled_balances(0.055, 358, 400000000)

In [ ]:
df.head()

In [ ]:
df = pd.DataFrame(data=np.zeros([nper + 1, 1]), columns=['scheduled_balance'],
                      index=[np.arange(0, nper + 1)])
df.loc[0, 'scheduled_balance'] = pv
df.loc[1:,'interest'] = np.ipmt(cpn/12,per=nper,pv=df.loc[1:].shift(-1))

In [ ]:
import collateral_waterfall as cw
import prepayment_calcs as pc
import numpy as np
import pandas as pd

In [ ]:
a=pc.cpr_curve_creator()

In [ ]:
factor = []
factor.append(1-pc.SMM(a[0]))

for i in range(1,len(a)):
    factor.append(factor[i-1] * (1-pc.SMM(a[i])))

In [ ]:
sched = cw.schedule_of_ending_balances(0.09, 360, 200000)['scheduled_balance']

In [ ]:
sched.head()

In [ ]:
waterfall = cw.create_waterfall(original_balance=4e8, pass_thru_cpn=0.055,
                                psa_speed=0,wam=360, wac=0.055)

In [ ]:
waterfall.tail()

In [ ]:
cpr = pc.cpr_curve_creator()

In [ ]:
smm = [0.01]*360

In [ ]:
age = 361-len(sched)
ab = sched['scheduled_balance'][:-1] * (1-0.01)

In [ ]:
ab.head()

In [ ]:
sched['scheduled_balance'][:-1].tail()

In [ ]:
import collateral_waterfall as cw
import prepayment_calcs as pc
import numpy as np
import pandas as pd

scheduled_ending = cw.schedule_of_ending_balances(0.09, 
                                       360, 
                                       150000)['scheduled_balance']

smm = [.01]

actual_ending = cw.actual_balances(scheduled_ending, smm)
df = pd.DataFrame(
    data=[scheduled_ending, actual_ending]).T.fillna(0)
df.columns=['scheduled','actual']
df = df.applymap('${:,.2f}'.format)
print(df.head())

cpr = pc.cpr_curve_creator()
smm = [pc.SMM(.10)]

actual_ending = cw.actual_balances(scheduled_ending, smm)
df = pd.DataFrame(
    data=[scheduled_ending, actual_ending]).T.fillna(0)
df.columns=['scheduled','actual']
df = df.applymap('${:,.2f}'.format)
print(df.head())

In [ ]:
pc.SMM(0.4) * 1.5

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
waterfall = cw.create_waterfall(original_balance=200000, 
                    psa_speed=3, 
                    pass_thru_cpn=0.075, 
                    wac=0.075,
                    wam=360)

In [ ]:
waterfall = {}

figure, axes = plt.subplots()
for i in range(15):
    for j in range(4):
        waterfall[i,j] = cw.create_waterfall(original_balance=200000, 
                        psa_speed=i + (j * 0.25), 
                        pass_thru_cpn=0.075, 
                        wac=0.075,
                        wam=360)
        waterfall[i,j].beginning_balance.plot(ax=axes)
plt.show()

In [ ]:
def arm_coupons(rate_curve, 
                gross_margin, 
                total_fees, 
                initial_coupon,
                periodic_cap):
    df = pd.DataFrame(index=pd.Index(range(len(rate_curve))),
                     columns=['rates','Gross','Net'])
    df.rates=rate_curve
    df.loc[0,'Gross'] = initial_coupon
    
    for i in range(1,df.index.max()+1):
        if df.rates[i] + 1.75 <= df.Gross[i-1] + 1:
            df.loc[i, 'Gross'] = df.rates[i] + 1.75 
        else:
            df.loc[i, 'Gross'] = df.Gross[i-1] + 1 
    
    df.loc[:,'Net'] = df.Gross - total_fees
    
    return df

In [ ]:
rates = [None,8.2,5.,5.75,4.]
df = arm_coupons(rates,1.75,0.65,5.1,1)

In [ ]:
df

In [ ]:
df.plot()
plt.show()

In [ ]:
1e6 * ((1.085) ** (14/365) - 1)

In [ ]:
mey = np.irr([-99,5,105])
mey = 0.0
bey = 2 * ((1+mey/12)**6-1)

In [ ]:
print('%f,%f' % (mey,bey))

In [ ]:
time = np.array([1,2,3,4,5])
principal = np.array([5,10,20,40,80])

In [ ]:
WAL = ((time * principal) / principal.sum()).sum()

In [ ]:
WAL

In [ ]:
def macauley_duration(price, times, cash_flows, yld):
    return (1 / price) * ((times * cash_flows) /
                        (1 + yld/2)**(2*times)).sum()


def modified_duration(duration, yld):
    return duration / (1 + yld/2)


def effective_duration(price, delta_down_price, delta_up_price, yld_delta):
    return (-100/price) * ((delta_down_price - delta_up_price) / (2 * yld_delta))

In [ ]:
duration = macauley_duration(price,times,cash_flows,yld)
modified_duration = modified_duration(duration,yld)

In [ ]:
modified_duration

In [ ]:
effective_duration(100,99.75,100.25,.01)

In [ ]:
speeds = [.5,1,2,5]
curves = [pc.cpr_curve_creator()*np.array(speed) for speed in speeds]

In [ ]:
fig, ax = plt.subplots()
for curve in curves:
    print(ax.plot(curve))

In [ ]:
plt.show()

In [ ]:
def schedule_of_ending_balance_percent_for_period(rate, nper, age):
    return (1. - ((((1 + rate) ** age) - 1.) /
                  (((1 + rate) ** nper) - 1.)))

In [ ]:
def example_matrix_of_balance_outstanding_by_age_and_term():
    coupon = 0.08
    terms = [180,360]
    ages = [24,48,72,96,120,144,168]
    
    df = pd.DataFrame(index=ages, columns=[terms]).unstack()
    
    return results

In [ ]:
results = {}

coupon = 0.08/12.
terms = [180,360]
ages = [24,48,72,96,120,144,168]
    
for term in terms:
    for age in ages:
        results[term,age] = scheduled_balance_percent(
        coupon, term, age)

In [ ]:
results

In [ ]:
original_balance=150000
pass_thru_coupon=0.08
wac=0.08
wam=360

first_wf = cw.create_waterfall(original_balance=original_balance, 
                               pass_thru_cpn=pass_thru_coupon,
                              wac=wac,
                              wam=wam,
                              cpr_description='7')
first_wf['CPR']=first_wf['SMM'].apply(pc.cpr)
first_wf.head()

In [ ]:
original_balance=150000
pass_thru_coupon=0.08
wac=0.08
wam=360

second_wf = cw.create_waterfall(original_balance=original_balance, 
                               pass_thru_cpn=pass_thru_coupon,
                              wac=wac,
                              wam=wam,
                              psa_speed=3)
second_wf['CPR']=second_wf['SMM'].apply(pc.cpr)
second_wf.head()

In [ ]:
def example_matrix_of_balance_outstanding_by_age_and_coupon():
    coupon = list([0.06, 0.08, 0.10])
    age = list([60, 120, 180, 240, 300, 359])

    results = []
    for i in coupon:
        column = []
        for j in age:
            column.append(schedule_of_ending_balance_percent_for_period(i / 12, 360, j))
        results.append(column)

    df = pd.DataFrame(results).T
    df.columns = coupon
    df.index = age
    return df

In [13]:
import numpy as np
import pandas as pd
import bond_pricing as bp

In [14]:
bonds = pd.DataFrame([
    {'Face': 100, 'Maturity': 0.25, 'Coupon': 0, 'Price': 97.50},
    {'Face': 100, 'Maturity': 0.5, 'Coupon': 0, 'Price': 94.90},
    {'Face': 100, 'Maturity': 1.0, 'Coupon': 0, 'Price': 90.00},
    {'Face': 100, 'Maturity': 1.5, 'Coupon': 8, 'Price': 96.00},
    {'Face': 100, 'Maturity': 2.0, 'Coupon': 12, 'Price': 101.60}
])

bonds['coupon_freq'] = 1

bonds = bp.BondPricing(bonds).bonds

In [15]:
bonds = pd.DataFrame([
    {'Face': 100, 'Maturity': 1., 'Coupon': 7, 'Price': 100.},
    {'Face': 100, 'Maturity': 2., 'Coupon': 8, 'Price': 100.},
    {'Face': 100, 'Maturity': 3., 'Coupon': 9, 'Price': 100.},
    {'Face': 100, 'Maturity': 4., 'Coupon': 10, 'Price': 100.}
])

bonds['coupon_freq'] = 1

In [16]:
bonds

,Coupon,Face,Maturity,Price,coupon_freq
0,7,100,1.0,100.0,1
1,8,100,2.0,100.0,1
2,9,100,3.0,100.0,1
3,10,100,4.0,100.0,1


In [126]:
bonds = pd.DataFrame([
    {'Maturity': 1., 'Yield': 7.},
    {'Maturity': 2., 'Yield': 8.},
    {'Maturity': 3., 'Yield': 9.},
    {'Maturity': 4., 'Yield': 10.},
])

In [214]:
def spot_from_par(bonds):
    bonds.loc[0,'spot_rate'] = bonds.loc[0,'Yield']
    for i in np.arange(1,len(bonds)):
        result = 0
        for j in range(i):
            #print(i,j)
            print('{0:.10f} / ( 1 + {1:.10f}/100)**({2:.0f})'.format(bonds.loc[i,'Yield'],
                                                                       bonds.loc[j,'spot_rate'],
                                                                       j+1))
            a = bonds.loc[i,'Yield'] / (1. + bonds.loc[j,'spot_rate']/100.)**(j + 1.)
            result += a
        bonds.loc[i,'spot_rate'] = (((100.+bonds.loc[i,'Yield']) / (100. - result)) ** (1./ (i+1.)) - 1.) * 100
    return bonds

In [215]:
a = spot_from_par(bonds)

8.0000000000 / ( 1 + 7.0000000000/100)**(1)
9.0000000000 / ( 1 + 7.0000000000/100)**(1)
9.0000000000 / ( 1 + 8.0403964854/100)**(2)
10.0000000000 / ( 1 + 7.0000000000/100)**(1)
10.0000000000 / ( 1 + 8.0403964854/100)**(2)
10.0000000000 / ( 1 + 9.1252553099/100)**(3)


In [216]:
a

,Maturity,Yield,spot_rate
0,1.0,7.0,7.000000
1,2.0,8.0,8.040396
2,3.0,9.0,9.125255
3,4.0,10.0,10.272363


In [190]:
9.0000000000 / ( 1 + 8.0403964854/100.)**(1 + 1)

7.710280373834972

In [203]:
c ** .5

1.080403964854224

In [166]:
9/b + 9.0000000000 / ( 1 + 8.0403964854/100)**(1 + 1)

16.121495327106

In [64]:
b= 1.07

In [65]:
c = (108 / (100 - (8 / b)))

In [66]:
d = (109 / (100 - (9/c + 9/b)))

In [75]:
e = (110 / (100 - (10/d + 10/c + 10/b)))

In [77]:
a

,Maturity,Yield,spot_rate
0,1.0,7.0,7.000000
1,2.0,8.0,8.040396
2,3.0,9.0,9.127757
3,4.0,10.0,10.284585


In [76]:
print(b, c**.5, d**(1/3),e**.25 )

1.07 1.080403964854224 1.0912525530985964 1.102723634972019


In [11]:
import collateral_waterfall as cw
import prepayment_calcs as pc
import bond_pricing as bp
import scipy.interpolate as interpolate

In [15]:
spot_curve = bp.BondPricing.spot_from_par()

8.0000000000 / ( 1 + 7.0000000000/100)**(1)
9.0000000000 / ( 1 + 7.0000000000/100)**(1)
9.0000000000 / ( 1 + 8.0403964854/100)**(2)
10.0000000000 / ( 1 + 7.0000000000/100)**(1)
10.0000000000 / ( 1 + 8.0403964854/100)**(2)
10.0000000000 / ( 1 + 9.1252553099/100)**(3)


In [108]:
def forward_rates(spot_curve):
    maturities = spot_curve['Maturity']
    spots = spot_curve['spot_rate']
    
    if len(maturities) != len(spots):
        raise AttributeError('maturity and rate vector lengths must match')

    size = len(spots) -1
    forward = maturities[1:] - maturities[0]
    forward.name = 'Forward'
    print(forward)
    frame = pd.DataFrame(np.full((size,size),np.nan),columns=forward,index=maturities[:-1])

    for i in range(len(maturities)-1):
        for j in range(1,len(maturities)):
            #print(i,j)
            try:
                r2 = spots[i+j] / 100
                r1 = spots[i] / 100
                t2 = maturities[i+j]
                t1 = maturities[i]

                forward_rate = ((((1 + r2)**t2) / ((1 + r1)**t1)) ** (1/(t2-t1)) - 1) * 100
                frame.loc[t2-t1, t1] = forward_rate
                #print('r2: {}\nr1: {}\nt2: {}\nt1:{}'.format(r2,r1,t2,t1))
            except:
                pass
                #print('loop')

    return frame

In [109]:
forwards = forward_rates(spot_curve)

1    1.0
2    2.0
3    3.0
Name: Forward, dtype: float64


In [110]:
spot_curve2 = spot_curve.copy()
spot_curve2.loc[2,'spot_rate'] = 11
spot_curve2.loc[3,'Maturity'] = 10
forwards = forward_rates(spot_curve2)

1    1.0
2    2.0
3    9.0
Name: Forward, dtype: float64


In [103]:
forwards

Forward,1.0,2.0,9.0,3.0
Maturity,,,,
1.0,9.090909,17.164650,NaN,NaN
2.0,13.055730,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN
9.0,10.642082,NaN,NaN,NaN
8.0,NaN,10.837523,NaN,NaN
7.0,NaN,NaN,NaN,9.961981


In [1]:
import xlwings as xw
import collateral_waterfall as cw
wb = xw.Book('C:/Users/Adam/Desktop/mbs waterfall.xlsx')

In [14]:
%%timeit

sheet1 = wb.sheets(1)
balance = sheet1.range('B1').value
WAM = int(sheet1.range('B2').value)
WAC = sheet1.range('B3').value
ptc = sheet1.range('B4').value
speed = sheet1.range('B5').value
cpr = sheet1.range('B6').value

anchor = sheet1.range('A9')

waterfall = cw.create_waterfall(original_balance=balance,
                               wam=WAM,
                               wac=WAC,
                               pass_thru_cpn=ptc,
                               psa_speed=speed,
                               cpr_description=cpr)
anchor.expand().value = ""
anchor.value = waterfall

1 loop, best of 3: 466 ms per loop


In [28]:
sheet1 = wb.sheets(1)
    values = sheet1.range('B1:B6').value

    balance = values[0]
    WAM = int(values[1])
    WAC = values[2]
    ptc = values[3]
    speed = values[4]
    cpr = values[5]

    anchor = sheet1.range('A9')

    waterfall = cw.create_waterfall(original_balance=balance,
                                   wam=WAM,
                                   wac=WAC,
                                   pass_thru_cpn=ptc,
                                   psa_speed=speed,
                                   cpr_description=cpr)
    anchor.expand().value = ""
    anchor.value = waterfall

In [29]:
ax = waterfall['beginning_balance'].plot()

In [33]:
sheet1.pictures.add(ax, name='Adam',update=True)

TypeError: Don't know what to do with that image object